In [95]:
import pandas as pd
import tushare as ts

from pyecharts import options as opts
from pyecharts.charts import Liquid, Page, Gauge
from pyecharts.globals import SymbolType

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
#CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

from pyecharts.commons import utils

In [2]:
#导入 tushare 数据包，提取 A 股基本数据信息，取毛利率 gpr 和净利率 npr 的平均数，并分别以水球图和仪 表盘图表现出来

stock = ts.get_stock_basics()
stock.head(2)

,name,industry,area,pe,outstanding,totals,totalAssets,liquidAssets,fixedAssets,reserved,...,bvps,pb,timeToMarket,undp,perundp,rev,profit,gpr,npr,holders
code,,,,,,,,,,,,,,,,,,,,,
002255,海陆重工,专用机械,江苏,29.33,6.44,8.42,75.65,51.33,14.63,21.57,...,4.53,0.67,20080625,7.17,0.85,31.67,-48.14,19.09,3.60,46829.0
688166,博瑞医药,化学制药,江苏,332.24,0.37,4.10,9.19,5.80,2.20,3.02,...,3.11,22.18,20191108,1.52,0.37,0.00,0.00,62.03,20.35,26778.0


In [121]:
# 水球图
c = (
    Liquid()
    .add(
        "lq", 
        [stock.gpr.mean()/100], 
        shape='roundRect', 
        is_outline_show=False,
        label_opts=opts.LabelOpts(
            formatter = utils.JsCode(
                # 通过 JS 函数控制显示数字的精度 https://pyecharts.org/#/zh-cn/basic_charts?id=liquid%ef%bc%9a%e6%b0%b4%e7%90%83%e5%9b%be
                """function (param) {
                    return (Math.floor(param.value * 10000) / 100) + '%';
                }"""
            ),
        position = 'inside'
        )
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="A 股毛利率平均值"))
)

c.load_javascript()

In [122]:
c.render_notebook()

In [147]:
# 仪表盘
gg = (
    Gauge()
    .add(
        series_name = '仪表盘',
        data_pair=[('平均市盈率', round(stock.npr.mean(), 2))],
        min_ = -100
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="A 股市盈率"))
)

gg.render_notebook()

In [153]:
# 统计 A 股各行业公司数量，并跟进行业分布数据作词云图(阅读参考文章 4、5);

from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType,ThemeType

data = stock.groupby(['industry']).name.agg([('comp_cnt', 'count')])


c = (
        WordCloud(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC) )#设置风格
        .add("", 
             zip(data.index, data.comp_cnt), 
             shape='pentagon', # shape设置词云图轮廓，有 'circle', 'cardioid', 'diamond', 'triangle-forward', 'triangle', 'pentagon', 'star' 可选
            )
        .set_global_opts(title_opts=opts.TitleOpts(title="A 股公司数分布（按行业）"))
)

#c.load_javascript()
c.render_notebook()

In [154]:
# 用 tushare 提取茅台数据中的最高最低价，作双折线图，茅台股票代码 600519 (提示:提取单个股票信息 df = ts.get_hist_data('600519',start=' ',end=' ') )

df = ts.get_hist_data('600519')
df.head()

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20
date,,,,,,,,,,,,,
2020-02-17,1082.50,1096.19,1093.82,1082.40,27028.22,5.82,0.54,1093.618,1076.958,1081.321,31343.31,37574.23,40173.65
2020-02-14,1090.45,1093.51,1088.00,1083.11,23287.60,-3.00,-0.28,1088.152,1067.968,1082.255,32044.35,47215.70,40621.03
2020-02-13,1098.00,1113.89,1091.00,1088.01,30357.36,-6.27,-0.57,1085.752,1064.448,1082.990,33642.57,50233.78,41326.94
2020-02-12,1089.00,1098.79,1097.27,1085.88,28125.43,-0.73,-0.07,1081.752,1062.899,1082.847,37005.34,50818.05,41059.49
2020-02-11,1063.00,1099.68,1098.00,1062.80,47917.93,31.51,2.96,1072.296,1060.702,1082.710,40863.90,51292.91,42045.89


In [196]:
from pyecharts.faker import  Faker
from pyecharts.charts import Line

data = df.loc[:, ['high', 'low']].sort_index()

c = (
    Line()
    .add_xaxis(data.index.values.tolist())
    .add_yaxis(
                "最高", data.high,
                markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="max")])
              )
    .add_yaxis(
                "最低", data.low,
                markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="min")]))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="贵州茅台股价趋势"),
        yaxis_opts=opts.AxisOpts(
            min_=300,
        ),
        # 由于时间范围跨度大，使用 DataZoomOpts 可以设置 X 轴的窗口，便于观察一段时间的数据
        # https://pyecharts.org/#/zh-cn/global_options?id=datazoomopts%ef%bc%9a%e5%8c%ba%e5%9f%9f%e7%bc%a9%e6%94%be%e9%85%8d%e7%bd%ae%e9%a1%b9
        datazoom_opts = opts.DataZoomOpts()
    )
)

#c.load_javascript()
c.render_notebook()